# Reinforcement Learning - Cheatsheet

In [ ]:
# !pip install stable-baselines3[extra]

In [4]:
import os
import gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

## Cartpole

### Env

In [1]:
ENV_NAME = 'CartPole-v0'

#### Testing

In [5]:
env = gym.make(ENV_NAME)

episodes = 5
for ep in range(1, episodes+1):
    state = env.reset() # initial observations for the env
    done = False
    score = 0
    while not done:
        env.render()
        action = env.action_space.sample()
        # there are two spaces - action, observation
        # here, observation_space is [cart_pos, cart_vel, pole_angle, pole_angvel]
        # action space: 0 - left, 1 - right
        n_state, reward, done, info = env.step(action)
        # n_state is observation for next state
        score += reward
    
    print(f"Ep #{ep} - Score: {score}")
env.close()

Ep #1 - Score: 19.0
Ep #2 - Score: 20.0
Ep #3 - Score: 24.0
Ep #4 - Score: 41.0
Ep #5 - Score: 44.0


In [ ]:
env = gym.make(ENV_NAME)
env = DummyVecEnv([lambda: env])

### Model

There are two types of RL learning algorithms
1. Model Free
    - Uses only the current state
2. Model Bases
    - Tries to predict and use future states

#### Init

In [ ]:
LOG_PATH = os.path.join('training', 'logs', ENV_NAME)

In [ ]:
# new
# MlpPolicy - Multilayer Perceptron
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=LOG_PATH) # creating a new model

In [ ]:
# load
MODEL_PATH = os.path.join('training', 'saved_models', ENV_NAME, 'v1')
model = PPO.load(MODEL_PATH, env=env) # for loading saved model

In [ ]:
PPO??

#### Learning

In [ ]:
model.learn(total_timesteps=20000)

#### Saving

In [ ]:
model.save(MODEL_PATH)
del model

#### Eval

In [ ]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

In [ ]:
env.close()

In [ ]:
episodes = 5
for ep in range(1, episodes+1):
    obs = env.reset()
    done = False
    score = 0
    while not done:
        env.render()
        action, _ = model.predict(obs)
        # _ holds next_state which is None here
        # used in recurrent policies?
        obs, reward, done, info = env.step(action)
        score += reward
    print(f"Ep #{ep} - Score: {score}")
env.close()

##### Tensorboard

In [ ]:
# !tensorboard --logdir={LOG_PATH}

##### Callbacks

In [ ]:
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold

In [ ]:
stop_callback = StopTrainingOnRewardThreshold(reward_threshold=200, verbose=1)

eval_callback = EvalCallback(
    env,
    callback_on_new_best=stop_callback,
    eval_freq=10000,
    best_model_save_path=os.path.join('training', 'saved_models', ENV_NAME, input()),
    verbose=1
)

model.learn(total_timesteps=20000, callable=eval_callback)

##### Custom Policy

In [ ]:
net_arch = [dict(pi=[128, 128, 128, 128], vf=[128, 128, 128, 128])]
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=LOG_PATH, policy_kwargs={'net_arch': net_arch})

model.learn(total_timesteps=20000, callback=eval_callback)

##### Another Algo

In [ ]:
from stable_baselines3 import DQN

model = DQN('MlpPolicy', env, verbose=1, tensorboard_log=LOG_PATH)

model.learn(total_timesteps=20000)